<a href="https://colab.research.google.com/github/Amna9191/DSA210_TermProject/blob/main/DSA210TermProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import os

# Function to load and parse the HTML file
def load_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')
    return soup

# Function to extract YouTube Music history
def extract_youtube_music_history(soup):
    music_history = []

    # Find all sections with YouTube Music data, using the header containing 'YouTube Music'
    music_entries = soup.find_all('div', class_='outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')

    for entry in music_entries:
        # Check if the entry is related to YouTube Music (has the header 'YouTube Music')
        header = entry.find('p', class_='mdl-typography--title')
        if header and 'YouTube Music' in header.text.strip():
            # Extract the song title
            song_tag = entry.find('a', href=True)
            if song_tag:
                song_title = song_tag.text.strip()
                song_url = song_tag.get('href')

                # Extract the artist/channel name
                channel_tag = song_tag.find_next('a', href=True)
                if channel_tag:
                    channel_name = channel_tag.text.strip()
                    channel_url = channel_tag.get('href')

                # Extract the timestamp
                timestamp_tag = entry.find_all('div', class_='content-cell mdl-cell mdl-cell--12-col mdl-typography--caption')
                if timestamp_tag:
                    timestamp = timestamp_tag[-1].text.strip()
                else:
                    timestamp = "No timestamp found"

                music_history.append({
                    'song_title': song_title,
                    'song_url': 'https://music.youtube.com' + song_url,
                    'channel_name': channel_name,
                    'channel_url': 'https://www.youtube.com' + channel_url,
                    'timestamp': timestamp
                })

    return music_history

# Function to display or process the data
def print_music_history(music_history):
    for entry in music_history:
        print(f"Song Title: {entry['song_title']}")
        print(f"Song URL: {entry['song_url']}")
        print(f"Channel Name: {entry['channel_name']}")
        print(f"Channel URL: {entry['channel_url']}")
        print(f"Watched at: {entry['timestamp']}")
        print("-" * 40)

if __name__ == "__main__":
    # Specify the path to your watch-history.html file
    file_path = "/content/watch-history.html"

    if not os.path.exists(file_path):
        print("The specified file does not exist.")
    else:
        # Load and parse the HTML file
        soup = load_html(file_path)

        # Extract YouTube Music history
        music_history = extract_youtube_music_history(soup)

        # Print out the music history
        if music_history:
            print_music_history(music_history)
        else:
            print("No YouTube Music history found.")


Streaming output truncated to the last 5000 lines.
Watched at: Products: YouTubeWhy is this here? This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  here.
----------------------------------------
Song Title: Mukhda Vekh Ke
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=ZxiCg4p5eXs
Channel Name: Mika Singh - Topic
Channel URL: https://www.youtube.comhttps://www.youtube.com/channel/UCmIJDVHH-W5eILcGPos_rKA
Watched at: Products: YouTubeWhy is this here? This activity was saved to your Google Account because the following settings were on: YouTube watch history. You can control these settings  here.
----------------------------------------
Song Title: The Hook Up Song
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=JXZHzxusjPk
Channel Name: Vishal - Shekhar - Topic
Channel URL: https://www.youtube.comhttps://www.youtube.com/channel/UCWXJyrk1diAufFuTHCTfQ2A
W

In [9]:
# Extract timestamp as well

from bs4 import BeautifulSoup
import os

# Function to load and parse the HTML file
def load_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')
    return soup

# Function to extract YouTube Music history
def extract_youtube_music_history(soup):
    music_history = []

    # Find all sections with YouTube Music data
    music_entries = soup.find_all('div', class_='outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')

    for entry in music_entries:
        # Check if the entry is related to YouTube Music (has the header 'YouTube Music')
        header = entry.find('p', class_='mdl-typography--title')
        if header and 'YouTube Music' in header.text.strip():
            # Extract the song title
            song_tag = entry.find('a', href=True)
            if song_tag:
                song_title = song_tag.text.strip()
                song_url = song_tag.get('href')

                # Extract the artist/channel name
                channel_tag = song_tag.find_next('a', href=True)
                if channel_tag:
                    channel_name = channel_tag.text.strip()
                    channel_url = channel_tag.get('href')

                # Extract the timestamp
                timestamp_tag = entry.find('div', class_='content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1')
                if timestamp_tag:
                    timestamp_full = timestamp_tag.text.strip()
                    # Add a space before the month name in the timestamp
                    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
                    for month in months:
                        if month in timestamp_full:
                            timestamp_full = timestamp_full.replace(month, f" {month}", 1)  # Add space before the month
                            break
                    timestamp = timestamp_full.strip()
                else:
                    timestamp = "No timestamp found"

                music_history.append({
                    'song_title': song_title,
                    'song_url': 'https://music.youtube.com' + song_url,
                    'channel_name': channel_name,
                    'channel_url': 'https://www.youtube.com' + channel_url,
                    'timestamp': timestamp
                })

    return music_history


# Function to display or process the data
def print_music_history(music_history):
    for entry in music_history:
        print(f"Song Title: {entry['song_title']}")
        print(f"Song URL: {entry['song_url']}")
        print(f"Channel Name: {entry['channel_name']}")
        print(f"Channel URL: {entry['channel_url']}")
        print(f"Watched at: {entry['timestamp']}")
        print("-" * 40)

if __name__ == "__main__":
    # Specify the path to your watch-history.html file
    file_path = "/content/watch-history.html"

    if not os.path.exists(file_path):
        print("The specified file does not exist.")
    else:
        # Load and parse the HTML file
        soup = load_html(file_path)

        # Extract YouTube Music history
        music_history = extract_youtube_music_history(soup)

        # Print out the music history
        if music_history:
            print_music_history(music_history)
        else:
            print("No YouTube Music history found.")


Streaming output truncated to the last 5000 lines.
Watched at: Watched Udd Jaa Kaale Kaava [Climax Version]Mithoon - Topic Sep 10, 2023, 9:00:41 PM GMT+03:00
----------------------------------------
Song Title: Mukhda Vekh Ke
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=ZxiCg4p5eXs
Channel Name: Mika Singh - Topic
Channel URL: https://www.youtube.comhttps://www.youtube.com/channel/UCmIJDVHH-W5eILcGPos_rKA
Watched at: Watched Mukhda Vekh KeMika Singh - Topic Sep 10, 2023, 4:11:17 PM GMT+03:00
----------------------------------------
Song Title: The Hook Up Song
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=JXZHzxusjPk
Channel Name: Vishal - Shekhar - Topic
Channel URL: https://www.youtube.comhttps://www.youtube.com/channel/UCWXJyrk1diAufFuTHCTfQ2A
Watched at: Watched The Hook Up SongVishal - Shekhar - Topic Sep 10, 2023, 4:10:43 PM GMT+03:00
----------------------------------------
Song Title: Mumbai Dilli Di Kudiyaan
Song URL: https://musi

In [16]:
# Saved extracted data into a csv file

import csv
from bs4 import BeautifulSoup
import os
import re

# Function to export music history to a CSV file
def save_music_history_to_csv(music_history, output_file='music_history.csv'):
    # Define the header for the CSV file
    headers = ['Song Title', 'Song URL', 'Artist', 'Date', 'Time']  # Added 'Date' and 'Time'

    # Write data to the CSV file
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()  # Write the header row
        for entry in music_history:
            writer.writerow({
                'Song Title': entry['song_title'],
                'Song URL': entry['song_url'],
                'Artist': entry['channel_name'],  # Use cleaned artist name
                'Date': entry['date'],            # Add the date column
                'Time': entry['time']             # Add the time column
            })

    print(f"Music history successfully exported to {output_file}")

# Function to load and parse the HTML file
def load_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'lxml')
    return soup

# Function to extract YouTube Music history
def extract_youtube_music_history(soup):
    music_history = []

    # Find all sections with YouTube Music data
    music_entries = soup.find_all('div', class_='outer-cell mdl-cell mdl-cell--12-col mdl-shadow--2dp')

    for entry in music_entries:
        # Check if the entry is related to YouTube Music (has the header 'YouTube Music')
        header = entry.find('p', class_='mdl-typography--title')
        if header and 'YouTube Music' in header.text.strip():
            # Extract the song title
            song_tag = entry.find('a', href=True)
            if song_tag:
                song_title = song_tag.text.strip()
                song_url = song_tag.get('href')

                # Extract the artist name (cleaned channel name)
                channel_tag = song_tag.find_next('a', href=True)
                if channel_tag:
                    channel_name = channel_tag.text.strip().replace(" - Topic", "")  # Remove " - Topic"

                # Extract and clean the timestamp
                timestamp_tag = entry.find('div', class_='content-cell mdl-cell mdl-cell--6-col mdl-typography--body-1')
                if timestamp_tag:
                    timestamp_full = timestamp_tag.text.strip()
                    # Use regex to extract the proper timestamp
                    match = re.search(r'(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{1,2},\s\d{4},\s[\d:]+(?:\s[AP]M)?\sGMT[+\-]\d{2}:\d{2}', timestamp_full)
                    if match:
                        timestamp = match.group()  # Extract the cleaned timestamp
                        # Ensure the year is part of the date
                        date_time_parts = timestamp.split(", ", 2)  # Split into [Month Day, Year, Time]
                        date = f"{date_time_parts[0]}, {date_time_parts[1]}"  # Combine Month Day and Year
                        time = date_time_parts[2]  # Time portion
                    else:
                        timestamp = "No timestamp found"
                        date = "No date found"
                        time = "No time found"
                else:
                    timestamp = "No timestamp found"
                    date = "No date found"
                    time = "No time found"

                music_history.append({
                    'song_title': song_title,
                    'song_url': 'https://music.youtube.com' + song_url,
                    'channel_name': channel_name,  # Renamed as artist
                    'date': date,                 # Save date separately
                    'time': time                  # Save time separately
                })

    return music_history

# Function to display or process the data
def print_music_history(music_history):
    for entry in music_history:
        print(f"Song Title: {entry['song_title']}")
        print(f"Song URL: {entry['song_url']}")
        print(f"Artist: {entry['channel_name']}")  # Renamed as artist
        print(f"Date: {entry['date']}")            # Print date separately
        print(f"Time: {entry['time']}")            # Print time separately
        print("-" * 40)

if __name__ == "__main__":
    # Specify the path to your watch-history.html file
    file_path = "/content/watch-history.html"

    if not os.path.exists(file_path):
        print("The specified file does not exist.")
    else:
        # Load and parse the HTML file
        soup = load_html(file_path)

        # Extract YouTube Music history
        music_history = extract_youtube_music_history(soup)

        # Print out the music history
        if music_history:
            print_music_history(music_history)

            # Save to a CSV file
            save_music_history_to_csv(music_history, output_file='music_history.csv')
        else:
            print("No YouTube Music history found.")


Streaming output truncated to the last 5000 lines.
----------------------------------------
Song Title: Mukhda Vekh Ke
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=ZxiCg4p5eXs
Artist: Mika Singh
Date: Sep 10, 2023
Time: 4:11:17 PM GMT+03:00
----------------------------------------
Song Title: The Hook Up Song
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=JXZHzxusjPk
Artist: Vishal - Shekhar
Date: Sep 10, 2023
Time: 4:10:43 PM GMT+03:00
----------------------------------------
Song Title: Mumbai Dilli Di Kudiyaan
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=UrOiwW4Rneg
Artist: Vishal - Shekhar
Date: Sep 10, 2023
Time: 4:07:12 PM GMT+03:00
----------------------------------------
Song Title: GHAGRA
Song URL: https://music.youtube.comhttps://music.youtube.com/watch?v=58jgsk7i3gQ
Artist: Rekha Bhardwaj
Date: Sep 10, 2023
Time: 4:05:27 PM GMT+03:00
----------------------------------------
Song Title: Udd Jaa Kaale Kaava 

In [17]:
import csv
from datetime import datetime

# Function to filter songs by date range
def filter_songs_by_date(csv_file, start_date, end_date, output_file='filtered_songs.csv'):
    filtered_songs = []

    # Read the CSV file
    with open(csv_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Convert the 'Date' column to a datetime object
            try:
                song_date = datetime.strptime(row['Date'], '%b %d, %Y')
                # Check if the date falls within the range
                if start_date <= song_date <= end_date:
                    filtered_songs.append(row)
            except ValueError:
                print(f"Skipping invalid date: {row['Date']}")

    # Write the filtered songs to a new CSV file
    if filtered_songs:
        with open(output_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
            writer.writeheader()  # Write the header row
            writer.writerows(filtered_songs)
        print(f"Filtered songs saved to {output_file}")
    else:
        print("No songs found in the specified date range.")

# Main execution
if __name__ == "__main__":
    # Specify the path to your CSV file
    csv_file = 'music_history.csv'

    # Define the date range
    start_date = datetime(2024, 9, 1)   # Start of September 2024
    end_date = datetime(2024, 12, 31)  # End of December 2024

    # Call the function to filter songs
    filter_songs_by_date(csv_file, start_date, end_date, output_file='filtered_songs.csv')


Filtered songs saved to filtered_songs.csv
